
❗ Оставлены рекомандации ревьюера


# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Кроме того, нужно корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?" data-toc-modified-id="Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [ ]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

<div class="alert alert-success">
<b>✔️ Комментарий ревьювера:</b> Необходимые библиотеки импортированы, ничего лишнего.
    
</div>

In [ ]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('C:\\Users\\HARD_REBOOT\\Downloads\\insurance.csv')

In [ ]:
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


* Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
* Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [ ]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные выглядят адекватно и в обработке не нуждаются. Очевидно странных выбросов, к примеру, слишком маленького/большого возраста, неадекватно высоких зарплат и семей в 50 человек не видно, поэтому не будем избавляться от остальных выбивающихся значений, пусть модель учится их обрабатывать.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится

**Обоснование:**

1. Подставляем в формулу обучения вместо матрицы X произведение XM, где M — обратимая квадратная матрица со случайными числами:
  
$$
w_m = ((XM)^T XM)^{-1} (XM)^T y
$$
  
2. Преобразуем выражение, используя правила матричного умножения $ (AB)^T = B^T A^T $ и $ (AB)^{-1} = B^{-1} A^{-1} $ :
  
$$
w_m = (X^TM^T XM)^{-1} X^TM^T y
$$

3. M — обратимая матрица, следовательно, $ (M^T)^{-1}M^T $ даёт единичную матрицу. Умножение матрицы на обратную матрицу равно единичной матрице, а умножение любой матрицы на единичную равно этой самой матрице, поэтому снова можем преобразовать выражение:

$$
w_m = M^{-1} (X^TX)^{-1}X^Ty
$$

4. Снова преобразуем выражение, т.к. вторая его часть — это формула обучения:

$$
w_m = M^{-1}w
$$

5. Подставим теперь $ w_m $ в формулу предсказания (матрицу тоже меняем на M):

$$
a_m = XMM^{-1}w
$$

5. $MM^{-1}$ — снова единичная матрица, умножение на неё равно умножаемой матрице. Преобразовываем выражение:

$$
a_m = Xw
$$

6. Следовательно $a_m = a$, что и требовалось доказать

### Вывод

Мы доказали, что при умножении матрицы признаков на некую обратимую матрицу предсказания модели останутся такими же, как при исходной матрице.

## Алгоритм преобразования

**Алгоритм**

1. Выделим из исходных данных признаки: целевой и фичи
2. Создадим обратимую квадратную матрицу со случайными числами размерностью 4x4, т.к. у нас 4 признака
3. Шифруем фичи способом умножения на обратимую матрицу

<div class="alert alert-success">
<b>✔️ Комментарий ревьювера:</b> Можно добавить пункт 4 — сравнение метрик r2.

</div>

In [ ]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [ ]:
random_matrix = np.random.normal(0, 1, size=(4, 4))

Проверим, что матрица обратима

In [ ]:
np.linalg.inv(random_matrix)

array([[-0.15322401, -0.61505429, -0.49127374, -0.40116523],
       [-0.72399782, -0.02741325, -1.38618345, -1.12388088],
       [ 0.62931673,  0.05228831, -0.00399652,  0.39646498],
       [ 0.28265313,  0.00805019,  0.08182665, -0.6066265 ]])

In [ ]:
encrypted_features = features.dot(random_matrix)

In [ ]:
encrypted_features

,0,1,2,3
0,-2606.719711,-32991.911545,-51652.988082,29087.261044
1,-2020.974948,-25267.535263,-39570.576352,22285.701593
2,-1122.709647,-13961.496163,-21867.430325,12316.937179
3,-2169.624470,-27745.135428,-43429.450271,24450.546236
4,-1382.124394,-17356.856852,-27178.551258,15307.864411
...,...,...,...,...
4995,-1873.794459,-23747.183730,-37178.314147,20933.486445
4996,-2738.799910,-34859.807994,-54572.267137,30727.293577
4997,-1768.563651,-22553.713759,-35305.153042,19877.003986
4998,-1710.231647,-21753.855401,-34053.062707,19173.216193


Сравним с исходными фичами:

In [ ]:
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


**Обоснование**

В предыдущем пункте мы теоретически доказали, что $a_m = a$, теперь подставим в формулы наши данные

Рассчитаем вектор весов для изначальной матрицы данных по формуле $w = (X^T X)^{-1} X^T y$

<div class="alert alert-warning">
<b>⚠️ Комментарий ревьювера:</b> Предпочтительнее использовать оператор @ при умножении матриц. Он строже, чем np.dot и допускает только матричное умножение.
    
</div>

In [ ]:
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)

Вычисляем вектор предсказаний $a$ по формуле $a = Xw$

In [ ]:
a = features.dot(w)

Вычисляем то же самое только уже для зашифрованной матрицы

In [ ]:
w_m = np.linalg.inv(encrypted_features.T.dot(encrypted_features)).dot(encrypted_features.T).dot(target)

a_m = encrypted_features.dot(w_m)

Посмотрим на сумму предсказаний в обоих случаях, для удобства сравнения округлив значение до 4 знаков после запятой

In [ ]:
a.sum().round(4)

879.1347

In [ ]:
a_m.sum().round(4)

879.1347

Разницы между предсказаниями нет. Если есть, то она очень минимальна и при округлении пропадает совсем

## Проверка алгоритма

Пишем функцию, которая на вход берёт матрицу признаков и целевой признак, создаёт модель линейной регрессии, обучается, делает предсказание.   
Выводить на экран будет метрику R2, округлённую до 4 знаков после запятой

In [ ]:
def lin_r(features, target):
    model = LinearRegression()
    model.fit(features, target)
    predictions = model.predict(features)


    print('R2:','{:.4f}'.format(r2_score(target,predictions)))

Применяем функцию на исходных данных:

In [ ]:
lin_r(features, target)

R2: 0.4249


Теперь на зашифрованных:

In [ ]:
lin_r(encrypted_features, target)

R2: 0.4249


<div class="alert alert-success">
<b>✔️ Комментарий ревьювера:</b> Качество предсказания действительно не изменяется, результат достигнут!
    
Еще могу порекомендовать вот такой материал:
    
https://habr.com/ru/post/479398/

https://habr.com/ru/company/vk/blog/478514/ Применение зашифрованных данных для машинного обучения

https://www.amazon.science/blog/machine-learning-models-that-act-on-encrypted-data#:~:text=Privacy%2Dpreserving%20machine%20learning%20 Privacy-preserving machine learning (PPML)

</div>

### Вывод

Качество моделей до и после преобразования не изменилось

<div class="alert alert-success">
<b>✔️ Комментарий ревьювера (v2):</b> На всякий случай, здесь я добавил дополнительные материалы по матрицам. Может пригодится =)
    
http://publish.sutd.ru/e_books/lin_alg_2013/html/matrix_33.html основные операции с матрицами    

https://python-scripts.com/numpy информация по numpy с примерами
    
https://xn--80apmoib.xn--p1ai/media/uploads/2019/07/18/2-2.Matricy_i_osnovnye_matrichnye_operacii.pdf матричные операции
    
https://ml-handbook.ru/chapters/linear_models/intro Можешь посмотреть вот здесь теорию по линейной регрессии и по матрицам в том числе.

Вот неплохая статья по основам матричных операций и линейным моделям, правда на английском языке:

https://jonathan-hui.medium.com/machine-learning-linear-algebra-a5b1658f0151

http://www.machinelearning.ru/wiki/images/b/bb/Sem08_factorizations.pdf по матричным разложениям
    
Как дополнение по матричным разложениями и теории машинного обучения можно посмотреть видеолекции К. Воронцова или Е. Соколова, они есть на Youtube.
    
</div>